In [28]:
import pandas as pd
import tqdm
import requests
from datasets import Dataset
import os
import getpass
from clearml import Task

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

In [26]:
os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your key: ')

In [34]:
# clear ml credentials

In [2]:
test_data_path = "/Users/ekaterina.baru/projects/SpicyRAG/data/v2_ragas_npa_dataset_firstPart.xlsx"

In [3]:
# project:
#   name: "SpicyRag"
# 
# database:
#   user: "user"
#   password: "password"
#   name: "spicyragdb"
#   host: "localhost"
#   port: 5432
# 
# logging:
#   level: "INFO"
# 
# server:
#   host: "0.0.0.0"
#   port: 8000
# 
# llm:
#   base_url: "https://integrate.api.nvidia.com/v1"
#   api_key: "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"
#   model: "meta/llama-3.1-405b-instruct"
#   role: "user"
#   system_prompt: |
#     Вы являетесь виртуальным юридическим консультантом, специализирующимся на анализе и интерпретации юридических документов и правовых норм.
#     Ваша задача — помочь пользователю, используя предоставленный контекст, чтобы дать точный и обоснованный ответ на его юридический вопрос.
#     Контекст может содержать выдержки из законов, нормативных актов, судебных решений, договоров или других юридических текстов.
# 
#     Инструкции:
#       1. Внимательно проанализируй контекст и определите, какие части контекста наиболее релевантны, приводи всю полезную информацию из контекста
#       2. Используй релевантные фрагменты контекста (не пиши к чему обращаешься, не упоминай пунктов) для создания полного и юридически обоснованного ответа. Приводи цитаты из контекста в соответствии
#       3. После ответа напиши вывод о написанной тобой информации, начиная со слова Таким образом
# 
#     ВАЖНО: не упоминай, что ты работаешь с контекстом! Отвечай структурно и по пунктам.
#   temperature: 0.15
#   top_p: 0.7
#   max_tokens: 10000
# 
# llm_rewriter:
#   base_url: "https://integrate.api.nvidia.com/v1"
#   api_key: "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"
#   model: "meta/llama-3.1-8b-instruct"
#   role: "user"
#   system_prompt: |
#     Вы - полезный помощник, который генерирует несколько поисковых запросов на основе одного входного запроса.
#     Выполните расширение запроса. Если существует несколько распространенных способов формулировки пользовательского вопроса или общих синонимов для ключевых слов в вопросе, обязательно верните несколько версий запроса с разными формулировками.
#     Если в тексте есть сокращения или слова, которые вам незнакомы, не пытайтесь их перефразировать.
#     Верните только 3 разных варианта вопроса.
#   temperature: 0.2
#   top_p: 0.7
#   max_tokens: 1024
# 
# embedding_model:
#   name: "paraphrase-MiniLM-L6-v2"
#   dimension: 384
# 
# data_sources:
#   excel_file: "./data/v2_ragas_npa_dataset_firstPart.xlsx"
#   text_file: "./data/hmao_npa.txt"
#   text_separator: "\n\n"
# 
# data_processing:
#   chunker:
#     py_class: interface.chunker.RecursiveCharacterTextSplitterChunker
#     kwargs:
#       chunk_size: 2048
#       chunk_overlap: 128
#       separators:
#         - '\n#{1,6} '
#         - '```\n'
#         - '\n\\*\\*\\*+\n'
#         - '\n---+\n'
#         - '\n___+\n'
#         - '\n\n'
#         - '\n'
#         - ' '
#         - ''
#         - ','
#         - '|'
#   top_k: 3
#   similarity_threshold: 0.7

In [4]:
def get_system_responses(questions):
    """
    Send requests to the /ask endpoint and collect responses.
    """
    responses = []
    for question in tqdm.tqdm(questions.itertuples()):
        try:
            response = requests.post("http://localhost:8000/ask/", json={"text": question[1]})
            response.raise_for_status()
            response_data = response.json()
            responses.append(
                {
                    "question": question[1],
                    "response": response_data["llm_response"],
                    "ground_truth": question[3],
                    "context": response_data["contexts"], 
                }
            )
        except Exception as e:
            print(f"Error while sending query: {question[1]}, error message: {e}")
            continue
    return responses

In [5]:
test_data = pd.read_excel(test_data_path)

In [6]:
system_responses = get_system_responses(test_data)

334it [1:31:54, 162.99s/it]

Error while sending query: Как зарплата влияет на мотивацию педагогов?, error message: string indices must be integers


500it [2:22:16, 17.07s/it] 


In [23]:
data_samples = {
    "question": [response["question"] for response in system_responses],
    "answer": [response["response"] for response in system_responses],
    "contexts": [response["context"] for response in system_responses],
    "ground_truth": [response["ground_truth"] for response in system_responses],
}
dataset = Dataset.from_dict(data_samples)

In [29]:
result = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/1996 [00:00<?, ?it/s]

Exception raised in Job[213]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-UZZpjHJPzrDNOTXMz3GCGmjK on tokens per min (TPM): Limit 200000, Used 195990, Requested 9581. Please try again in 1.671s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[219]: TimeoutError()
Exception raised in Job[655]: TimeoutError()
Exception raised in Job[225]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-UZZpjHJPzrDNOTXMz3GCGmjK on tokens per min (TPM): Limit 200000, Used 198034, Requested 9185. Please try again in 2.165s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[226]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt

{'context_precision': 0.9005, 'faithfulness': 0.4742, 'answer_relevancy': 0.4440, 'context_recall': 0.7680}

In [31]:
# {'context_precision': 0.9005, 'faithfulness': 0.4742, 'answer_relevancy': 0.4440, 'context_recall': 0.7680}

In [33]:

# Initialize the task
task = Task.init(
    project_name="SpicyRAG_experiments_tracker", 
    task_name="Exp_1_Baseline",  
    task_type=Task.TaskTypes.application  # Specify the task type
)

# Log each metric
for metric_name, metric_value in result.items():
    task.get_logger().report_scalar(
        title=metric_name,   # Metric name
        series="evaluation", # Group or series (optional but useful for grouping related metrics)
        value=metric_value,  # The value of the metric
        iteration=1          # The iteration number or epoch (set accordingly)
    )

# Close the task when done
task.close()

ClearML Task: created new task id=dae74f13bd464828ab4309bdf927ee03
2024-09-06 12:50:59,928 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/29330718a2a04a3582ac1092ef326bea/experiments/dae74f13bd464828ab4309bdf927ee03/output/log
2024-09-06 12:51:00,945 - clearml.Task - INFO - Waiting for repository detection and full package requirement analysis
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
2024-09-06 12:51:03,918 - clearml.Task - INFO - Finished repository detection and package analysis
